In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import tqdm 

In [2]:
import skimage, os
from skimage.morphology import ball, disk, dilation,binary_dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing, binary_opening 
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border, mark_boundaries
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.misc
from glob import glob
from skimage.io import imread


In [6]:
covidx_dir = '/media/ubuntu/MyHDataStor2/datasets/COVID-19/ICCV-MIA/'
covidx_img_dir= covidx_dir 
covidx_mask_dir= covidx_dir + 'mask/' 
data_list_dir = '/media/ubuntu/MyHDataStor2/products/COVID-19/ICCV-MAI/3D-CNN-BERT/BERT/datasets/settings/covid/'

print(covidx_dir)
print(covidx_img_dir)
print(covidx_mask_dir)
print(data_list_dir)

/media/ubuntu/MyHDataStor2/datasets/COVID-19/ICCV-MIA/
/media/ubuntu/MyHDataStor2/datasets/COVID-19/ICCV-MIA/
/media/ubuntu/MyHDataStor2/datasets/COVID-19/ICCV-MIA/mask/
/media/ubuntu/MyHDataStor2/products/COVID-19/ICCV-MAI/3D-CNN-BERT/BERT/datasets/settings/covid/


In [4]:
def seedfill(img):

  rows,cols = img.shape 

  img_floodfill = img.copy()
  mask = np.zeros((rows+2,cols+2), np.uint8)

  '''
  # Floodfill from point (0, 0)
  found  = False 
  for x in range(cols): 
    for y in range(rows):
      if img_floodfill[y,x] == 0:
        found = True 
        break  
    if found:
      break 

  #cv2.floodFill(img_floodfill, mask, (x,y), 255);
  ''' 
  

  cv2.floodFill(img_floodfill, mask, (0,0), 255);
  cv2.floodFill(img_floodfill, mask, (0,rows-1), 255);
  cv2.floodFill(img_floodfill, mask, (cols-1,0), 255);
  cv2.floodFill(img_floodfill, mask, (cols-1,rows-1), 255);

  return img_floodfill 


In [5]:
def body_contour(binary_image):
    """Helper function to get body contour"""
    _, contours, hier = cv2.findContours(binary_image, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    body_idx = np.argmax(areas)
    return contours[body_idx]


def auto_body_crop(image, plot=False):
    """Roughly crop an image to the body region"""
    # Create initial binary image
    filt_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    if plot == True:
        f, plots = plt.subplots(3, 1, figsize=(5, 40))        
        plots[0].axis('off')
        plots[0].imshow(filt_image, cmap=plt.cm.bone)     
    
    
    thresh = cv2.threshold(filt_image[filt_image > 0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    #print(thresh) 
    
    bin_image = np.uint8(filt_image > thresh)  
    #bin_image2 = np.uint8(filt_image > 240)         
    
    
    if plot == True:
        plots[1].axis('off')
        plots[1].imshow(bin_image, cmap=plt.cm.bone)     
    
    
    erode_kernel = np.ones((7, 7), dtype=np.uint8)
    bin_image = cv2.erode(bin_image, erode_kernel)
    
    #print(np.unique(bin_image))
    if plot == True:
        plots[2].axis('off')
        plots[2].imshow(bin_image, cmap=plt.cm.bone)     
        
    # Find body contour
    body_cont = body_contour(bin_image).squeeze()
    
    '''
    #find the lung rectangle        
    binary2 = ((1-bin_image)*255).astype(np.uint8) 
    kernel = np.ones((5,5), np.uint8) 
    binary2 = cv2.dilate(binary2, kernel, iterations=2) 
    binary2 = cv2.erode(binary2, kernel, iterations=2) 
    binary2 = seedfill(binary2) 
    binary2 = cv2.bitwise_not(binary2)    
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(binary2, cmap=plt.cm.bone) 
    '''    
    
    #print(body_cont)

    # Get bbox
    xmin = body_cont[:, 0].min()
    xmax = body_cont[:, 0].max() + 1
    ymin = body_cont[:, 1].min()
    ymax = body_cont[:, 1].max() + 1
       
    return bin_image, (xmin, ymin, xmax, ymax)


def auto_segment(image, plot=False):
    """Roughly crop an image to the body region"""
    # Create initial binary image
    filt_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    if plot == True:
        f, plots = plt.subplots(9, 1, figsize=(5, 40))        
        plots[0].axis('off')
        plots[0].imshow(filt_image, cmap=plt.cm.bone)     
    
    
    thresh = cv2.threshold(filt_image[filt_image > 0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    print(thresh) 
    
    bin_image = np.uint8(filt_image > thresh)  
    #bin_image2 = np.uint8(filt_image > 240)             
    
    if plot == True:
        plots[1].axis('off')
        plots[1].imshow(bin_image, cmap=plt.cm.bone)     
        
    erode_kernel = np.ones((7, 7), dtype=np.uint8)
    bin_image = cv2.erode(bin_image, erode_kernel)
    #print(np.unique(bin_image))
    if plot == True:
        plots[2].axis('off')
        plots[2].imshow(bin_image, cmap=plt.cm.bone)     
        
 
        
    # Find body contour
    body_cont = body_contour(bin_image).squeeze()
    
    #print(body_cont)

    # Get bbox
    xmin = body_cont[:, 0].min()
    xmax = body_cont[:, 0].max() + 1
    ymin = body_cont[:, 1].min()
    ymax = body_cont[:, 1].max() + 1
       
        
    bin_image_inv = (1 - bin_image)*255 
    bin_image_inv = seedfill(bin_image_inv)/255 
    cleared = clear_border(bin_image_inv)
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(cleared, cmap=plt.cm.bone)     
    
    label_image = label(cleared)
    if plot == True:
        plots[4].axis('off')
        plots[4].imshow(label_image, cmap=plt.cm.bone) 
        
    '''
    Step 4: Keep the labels with 2 largest areas.
    '''
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       label_image[coordinates[0], coordinates[1]] = 0

    binary = label_image > 0
    if plot == True:
        plots[5].axis('off')
        plots[5].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 5: Erosion operation with a disk of radius 2. This operation is 
    seperate the lung nodules attached to the blood vessels.
    '''
    selem = disk(7)
    binary = binary_erosion(binary, selem)
    if plot == True:
        plots[6].axis('off')
        plots[6].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 6: Closure operation with a disk of radius 10. This operation is 
    to keep nodules attached to the lung wall.
    '''
    selem = disk(10)
    binary = binary_closing(binary, selem)
    if plot == True:
        plots[7].axis('off')
        plots[7].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 7: Fill in the small holes inside the binary mask of lungs.
    '''

    binary1 = binary.copy() 

    edges = roberts(binary)
    binary = ndi.binary_fill_holes(edges)
    if plot == True:
        plots[8].axis('off')
        plots[8].imshow(binary, cmap=plt.cm.bone) 
    
    
    return binary, binary1, (xmin, ymin, xmax, ymax)


In [7]:
def refine_mask(m_full):
    
    mask2 = cv2.imread(m_full, cv2.IMREAD_UNCHANGED)
    mask2 = (mask2/255).astype(np.uint8) 
    
    selem = disk(10)
    mask2_2  = binary_closing(mask2, selem)                
    label_image = label(mask2_2)
    
    '''
    Step 4: Keep the labels with 2 largest areas.
    '''
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                    label_image[coordinates[0], coordinates[1]] = 0
    mask2_3 = label_image > 0               
    
    edges = roberts(mask2_3) 
    mask2_4 = ndi.binary_fill_holes(edges)
                
    return mask2,mask2_4

In [16]:
#subsets = ['val','train']  #for test dataset, UNet segmentation and refinement are combined. 
subsets = ['val']

from tqdm import tqdm

debug = False
split = 4 

for subset in subsets:     

    list_file = data_list_dir + '{}_rgb_split{}.txt'.format(subset,split) 
    lines = open(list_file,"r").readlines() 

    pbar = tqdm(total = len(lines))
    
    for l in lines:
        
        print(l)
        
        pbar.update()
        
        l_splits = l.split()  
                
        l_path = l_splits[0]        
        c = l_splits[1]
        first_slice = int(l_splits[2])
        last_slice = int(l_splits[3])

        count = 0 
        for f in range(first_slice,last_slice+1):
            
            count +=1 
            
            #if count>10:
            #    break 
            
            fn = "{}/{}.jpg".format(l_path,f)
            f_full = covidx_img_dir + fn
            m_full = covidx_img_dir + 'mask/' + fn
            m_full_new = m_full+'_refined.jpg'            
            #print(f_full,m_full,m_full_new)
            
            if not os.path.exists(m_full_new) :
                mask0,mask = refine_mask(m_full)                
                cv2.imwrite(m_full_new, (mask*255).astype(np.uint8) )

                if False: 
                    fig, (ax1,ax2) = plt.subplots(1,2)
                    ax1.imshow(mask0*255,cmap='gray')
                    ax2.imshow(mask*255,cmap='gray')                
                










  0%|          | 0/374 [00:00<?, ?it/s]

val/covid/ct_scan_0 covid 102 238 0 85 512 441

val/covid/ct_scan_1 covid 122 275 0 131 512 450

val/covid/ct_scan_2 covid 12 27 0 113 512 463

val/covid/ct_scan_3 covid 16 41 0 82 512 409

val/covid/ct_scan_4 covid 17 53 0 103 512 458

val/covid/ct_scan_5 covid 101 251 0 115 512 492

val/covid/ct_scan_6 covid 106 294 10 124 481 437

val/covid/ct_scan_7 covid 16 43 39 63 512 463

val/covid/ct_scan_8 covid 13 33 0 114 512 442

val/covid/ct_scan_9 covid 109 258 0 137 507 474

val/covid/ct_scan_10 covid 134 311 0 0 512 444

val/covid/ct_scan_11 covid 14 38 23 96 512 456

val/covid/ct_scan_12 covid 16 33 0 99 512 512

val/covid/ct_scan_13 covid 62 193 0 86 512 464

val/covid/ct_scan_14 covid 69 86 0 128 435 355

val/covid/ct_scan_15 covid 10 29 0 85 512 431

val/covid/ct_scan_16 covid 94 275 0 87 512 439

val/covid/ct_scan_17 covid 22 39 0 150 512 469

val/covid/ct_scan_18 covid 28 93 0 41 512 471

val/covid/ct_scan_19 covid 99 215 8 83 480 423

val/covid/ct_scan_20 covid 14 37 0 110 512 4









 48%|████▊     | 178/374 [00:00<00:00, 303.14it/s]







 80%|████████  | 300/374 [00:00<00:00, 391.30it/s]

val/non-covid/ct_scan_12 non-covid 10 33 0 78 512 490

val/non-covid/ct_scan_13 non-covid 10 24 0 61 512 444

val/non-covid/ct_scan_14 non-covid 102 331 20 119 497 431

val/non-covid/ct_scan_15 non-covid 38 221 0 37 512 477

val/non-covid/ct_scan_16 non-covid 44 67 0 102 512 490

val/non-covid/ct_scan_17 non-covid 113 338 0 111 503 475

val/non-covid/ct_scan_18 non-covid 15 34 0 63 512 447

val/non-covid/ct_scan_19 non-covid 75 253 0 65 512 469

val/non-covid/ct_scan_20 non-covid 97 320 0 75 512 481

val/non-covid/ct_scan_21 non-covid 95 311 0 23 512 442

val/non-covid/ct_scan_22 non-covid 79 217 0 76 502 429

val/non-covid/ct_scan_23 non-covid 74 280 7 135 512 482

val/non-covid/ct_scan_24 non-covid 106 253 32 71 477 414

val/non-covid/ct_scan_25 non-covid 66 205 0 95 512 385

val/non-covid/ct_scan_26 non-covid 109 297 0 86 512 408

val/non-covid/ct_scan_27 non-covid 84 298 11 107 466 442

val/non-covid/ct_scan_28 non-covid 71 231 11 74 502 445

val/non-covid/ct_scan_29 non-covid 95 2









100%|██████████| 374/374 [00:11<00:00, 391.30it/s]